In [1]:
import os
import xml
import pandas as pd
import numpy as np
import skimage.transform as skTrans
import nibabel as nib
import sys
import time
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [2]:
information_table=pd.DataFrame(None,columns=['Image ID','group', 'PATNO Visit'])

In [3]:
filepath='MRI_nobrain'
files= [filename.split('.')[0] for filename in os.listdir(filepath) if os.path.isfile(os.path.join(filepath,filename))]

In [4]:
files

['PDAA852XL7_PDBP_ImagingMR_1634919016780_PSP_021_1_sT1W_3D_TFE_SENSE_5_1',
 'PDAA852XL7_PDBP_ImagingMR_1634919016887_PSP_021_1_T2W_VISTA_HR_SENSE_9_1',
 'PDAA852XL7_PDBP_ImagingMR_1634921322116_PSP_021_2_sT1W_3D_TFE_SENSE_5_1',
 'PDAA852XL7_PDBP_ImagingMR_1634921322227_PSP_021_2_T2W_VISTA_HR_SENSE_10_1',
 'PDAD926WRZ_PDBP_ImagingMR_1634911430459_PDL_004_sT1W_3D_TFE_SENSE_7_1',
 'PDAD926WRZ_PDBP_ImagingMR_1634911430575_PDL_004_T2W_VISTA_HR_SENSE_8_1',
 'PDAD926WRZ_PDBP_ImagingMR_1634917064356_PDL_004_2_sT1W_3D_TFE_SENSE_6_1',
 'PDAD926WRZ_PDBP_ImagingMR_1634917064509_PDL_004_2_T2W_VISTA_HR_SENSE_10_1',
 'PDAH499RDX_PDBP_ImagingMR_1634915227941_PDL_031_1_sT1W_3D_TFE_SENSE_6_1',
 'PDAH499RDX_PDBP_ImagingMR_1634915228042_PDL_031_1_T2W_VISTA_HR_SENSE_13_1',
 'PDAH499RDX_PDBP_ImagingMR_1634917156717_PDL_031_2_sT1W_3D_TFE_SENSE_6_1',
 'PDAH499RDX_PDBP_ImagingMR_1634917156838_PDL_031_2_T2W_VISTA_HR_SENSE_13_1',
 'PDAK439ZK9_PDBP_ImagingMR_1634915138629_PDL_18_1_sT1W_3D_TFE_SENSE_6_1',
 'PDAK4

In [5]:
information_table['Image ID']=files

In [6]:
information_table

,Image ID,group,PATNO Visit
0,PDAA852XL7_PDBP_ImagingMR_1634919016780_PSP_02...,NaN,NaN
1,PDAA852XL7_PDBP_ImagingMR_1634919016887_PSP_02...,NaN,NaN
2,PDAA852XL7_PDBP_ImagingMR_1634921322116_PSP_02...,NaN,NaN
3,PDAA852XL7_PDBP_ImagingMR_1634921322227_PSP_02...,NaN,NaN
4,PDAD926WRZ_PDBP_ImagingMR_1634911430459_PDL_00...,NaN,NaN
...,...,...,...
708,PDZP161DT6_PDBP_ImagingMR_1634921352197_PSP_03...,NaN,NaN
709,PDZP161DT6_PDBP_ImagingMR_1634921352309_PSP_03...,NaN,NaN
710,PDZT233BE7_PDBP_ImagingMR_1634915165074_PDL_02...,NaN,NaN
711,PDZT233BE7_PDBP_ImagingMR_1634915165174_PDL_02...,NaN,NaN


In [7]:
query=pd.read_csv('query_result_PDBP_ImagingMR_2024-05-29T05-52-281521437268941414993.csv')

C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_23580\3253822051.py:1: DtypeWarning: Columns (13,20,21,43,47,51,55,57,58,61,63,64,66,67,68,71,75,76,77,82,127,128,129,130,131) have mixed types. Specify dtype option on import or set low_memory=False.
  query=pd.read_csv('query_result_PDBP_ImagingMR_2024-05-29T05-52-281521437268941414993.csv')


In [8]:
query

,Study ID,Dataset,PDBP_ImagingMR.Main.SiteName,PDBP_ImagingMR.Main.VisitTypPDBP,PDBP_ImagingMR.Main.VisitDate,PDBP_ImagingMR.Main.GUID,PDBP_ImagingMR.Main.ASSOCIATED GUID,PDBP_ImagingMR.Main.AgeYrs,PDBP_ImagingMR.Image Information.ImgStdyDateTime,PDBP_ImagingMR.Image Information.ImgFile,...,PDBP_ImagingMR.Magnetic Resonance Information.ImgBaseResolutionVal,PDBP_ImagingMR.Magnetic Resonance Information.ImgShotCt,PDBP_ImagingMR.Magnetic Resonance Information.ImgSignalAvgNum,PDBP_ImagingMR.Magnetic Resonance RF Coil.ImgRFCoilName,PDBP_ImagingMR.Magnetic Resonance RF Coil.ImgRFCoilChannelCt,PDBP_ImagingMR.Magnetic Resonance RF Coil.ImgRFCoilChannelOTH,PDBP_ImagingMR.Magnetic Resonance RF Coil.ImgRFCoilFunctTyp,PDBP_ImagingMR.Magnetic Resonance RF Coil.ImgRFCoilTyp,PDBP_ImagingMR.Magnetic Resonance RF Coil.ImgRFCoilOTH,PDBP_ImagingMR.Magnetic Resonance RF Coil.ImgRFCoilCat
0,202.0,PDBP-DATA0055727,University of Florida (Gainesville),12 months,NaN,PDAK439ZK9,NaN,50 - 59,NaN,/pdbp_imagingmr_output_1634917064356/PDAK439ZK...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,202.0,PDBP-DATA0055727,University of Florida (Gainesville),12 months,NaN,PDAK439ZK9,NaN,50 - 59,NaN,/pdbp_imagingmr_output_1634917064356/PDAK439ZK...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,235.0,PDBP-DATA0029690,University of Florida (Gainesville),Baseline,NaN,PDJW615BUW,NaN,70 - 79,2017-12-15T00:00:00Z,/pdbp_imagingmr_output_1539784418934/PDJW615BU...,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202.0,PDBP-DATA0055727,University of Florida (Gainesville),12 months,NaN,PDLC368NK0,NaN,50 - 59,NaN,/pdbp_imagingmr_output_1634917064356/PDLC368NK...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,202.0,PDBP-DATA0055727,University of Florida (Gainesville),12 months,NaN,PDLC368NK0,NaN,50 - 59,2013-07-11T10:57:37Z,/pdbp_imagingmr_output_1634917064356/PDLC368NK...,...,NaN,NaN,1.0,Dual coil,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6094,202.0,PDBP-DATA0039491,University of Florida,Baseline,NaN,PDNF936JW4,NaN,60 - 69,NaN,/RawData-David Vaillancourt-ImagingMR/PDNF936J...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6095,202.0,PDBP-DATA0039491,University of Florida,Baseline,NaN,PDNW868YJ6,NaN,60 - 69,NaN,/RawData-David Vaillancourt-ImagingMR/PDNW868Y...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6096,233.0,PDBP-DATA0061450,Cleveland Clinic,Baseline,NaN,NIHNE902MGFBR,NaN,60 - 69,2019-12-17T10:35:56Z,/pdbp_imagingmr_output_1656028633008/NIHNE902M...,...,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6097,233.0,PDBP-DATA0061450,Cleveland Clinic,Baseline,NaN,NIHNE902MGFBR,NaN,60 - 69,2019-12-17T10:35:56Z,/pdbp_imagingmr_output_1656028633008/NIHNE902M...,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
query[['Dataset','PDBP_ImagingMR.Main.VisitTypPDBP','PDBP_ImagingMR.Main.GUID','PDBP_ImagingMR.Image Information.ImgFile']]

,Dataset,PDBP_ImagingMR.Main.VisitTypPDBP,PDBP_ImagingMR.Main.GUID,PDBP_ImagingMR.Image Information.ImgFile
0,PDBP-DATA0055727,12 months,PDAK439ZK9,/pdbp_imagingmr_output_1634917064356/PDAK439ZK...
1,PDBP-DATA0055727,12 months,PDAK439ZK9,/pdbp_imagingmr_output_1634917064356/PDAK439ZK...
2,PDBP-DATA0029690,Baseline,PDJW615BUW,/pdbp_imagingmr_output_1539784418934/PDJW615BU...
3,PDBP-DATA0055727,12 months,PDLC368NK0,/pdbp_imagingmr_output_1634917064356/PDLC368NK...
4,PDBP-DATA0055727,12 months,PDLC368NK0,/pdbp_imagingmr_output_1634917064356/PDLC368NK...
...,...,...,...,...
6094,PDBP-DATA0039491,Baseline,PDNF936JW4,/RawData-David Vaillancourt-ImagingMR/PDNF936J...
6095,PDBP-DATA0039491,Baseline,PDNW868YJ6,/RawData-David Vaillancourt-ImagingMR/PDNW868Y...
6096,PDBP-DATA0061450,Baseline,NIHNE902MGFBR,/pdbp_imagingmr_output_1656028633008/NIHNE902M...
6097,PDBP-DATA0061450,Baseline,NIHNE902MGFBR,/pdbp_imagingmr_output_1656028633008/NIHNE902M...


In [10]:
PDBP_MRI_query=query[['Dataset','PDBP_ImagingMR.Main.VisitTypPDBP','PDBP_ImagingMR.Main.GUID','PDBP_ImagingMR.Image Information.ImgFile']]

In [11]:
PDBP_MRI_query['PDBP_ImagingMR.Image Information.ImgFile'][0].split('/')[-1].split('.')[0]

'PDAK439ZK9_PDBP_ImagingMR_1634917112740_PDL_018_2_sT1W_3D_TFE_SENSE_6_1'

In [12]:
PDBP_MRI_query['Image ID']=None

C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_23580\2677398304.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PDBP_MRI_query['Image ID']=None


In [13]:
for i in range(len(query)):
    if pd.isna(PDBP_MRI_query['PDBP_ImagingMR.Image Information.ImgFile'][i]):
        print(i)

3081
3243
3244
3245
5155
5156
5157
5970
5971
5972


In [14]:
for i in range(len(query)):
    try:
        PDBP_MRI_query['Image ID'][i]=PDBP_MRI_query['PDBP_ImagingMR.Image Information.ImgFile'][i].split('/')[-1].split('.')[0]
    except:
        print(i)

C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_23580\4054195095.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PDBP_MRI_query['Image ID'][i]=PDBP_MRI_query['PDBP_ImagingMR.Image Information.ImgFile'][i].split('/')[-1].split('.')[0]


3081
3243
3244
3245
5155
5156
5157
5970
5971
5972


In [15]:
PDBP_MRI_query

,Dataset,PDBP_ImagingMR.Main.VisitTypPDBP,PDBP_ImagingMR.Main.GUID,PDBP_ImagingMR.Image Information.ImgFile,Image ID
0,PDBP-DATA0055727,12 months,PDAK439ZK9,/pdbp_imagingmr_output_1634917064356/PDAK439ZK...,PDAK439ZK9_PDBP_ImagingMR_1634917112740_PDL_01...
1,PDBP-DATA0055727,12 months,PDAK439ZK9,/pdbp_imagingmr_output_1634917064356/PDAK439ZK...,PDAK439ZK9_PDBP_ImagingMR_1634917112837_PDL_01...
2,PDBP-DATA0029690,Baseline,PDJW615BUW,/pdbp_imagingmr_output_1539784418934/PDJW615BU...,PDJW615BUW_PDBP_ImagingMR_1539784418934
3,PDBP-DATA0055727,12 months,PDLC368NK0,/pdbp_imagingmr_output_1634917064356/PDLC368NK...,PDLC368NK0_PDBP_ImagingMR_1634917108372_PDL_01...
4,PDBP-DATA0055727,12 months,PDLC368NK0,/pdbp_imagingmr_output_1634917064356/PDLC368NK...,PDLC368NK0_PDBP_ImagingMR_1634917108479
...,...,...,...,...,...
6094,PDBP-DATA0039491,Baseline,PDNF936JW4,/RawData-David Vaillancourt-ImagingMR/PDNF936J...,PDNF936JW4_ImagingMR_1427505427566_MSA_035_1
6095,PDBP-DATA0039491,Baseline,PDNW868YJ6,/RawData-David Vaillancourt-ImagingMR/PDNW868Y...,PDNW868YJ6_ImagingMR_1428582370464_PDL_059_1
6096,PDBP-DATA0061450,Baseline,NIHNE902MGFBR,/pdbp_imagingmr_output_1656028633008/NIHNE902M...,NIHNE902MGFBR_PDBP_ImagingMR_1656038998839
6097,PDBP-DATA0061450,Baseline,NIHNE902MGFBR,/pdbp_imagingmr_output_1656028633008/NIHNE902M...,NIHNE902MGFBR_PDBP_ImagingMR_1656038973630


In [16]:
set(PDBP_MRI_query['PDBP_ImagingMR.Main.VisitTypPDBP'])

{'12 months',
 '18 months',
 '24 months',
 '36 months',
 '48 months',
 '6 months',
 'Baseline',
 nan}

In [17]:
M_set={'Baseline':'BL','6 months':'V02','12 months':'V04','18 months':'V05','24 months':'V06','36 months':'V08','48 months':'V10'}

In [18]:
PDBP_MRI_query['Visit']=None

C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_23580\628183333.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PDBP_MRI_query['Visit']=None


In [19]:
PDBP_MRI_query['Visit']=PDBP_MRI_query['PDBP_ImagingMR.Main.VisitTypPDBP'].map(M_set)

C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_23580\1672208881.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PDBP_MRI_query['Visit']=PDBP_MRI_query['PDBP_ImagingMR.Main.VisitTypPDBP'].map(M_set)


In [20]:
PDBP_MRI_query

,Dataset,PDBP_ImagingMR.Main.VisitTypPDBP,PDBP_ImagingMR.Main.GUID,PDBP_ImagingMR.Image Information.ImgFile,Image ID,Visit
0,PDBP-DATA0055727,12 months,PDAK439ZK9,/pdbp_imagingmr_output_1634917064356/PDAK439ZK...,PDAK439ZK9_PDBP_ImagingMR_1634917112740_PDL_01...,V04
1,PDBP-DATA0055727,12 months,PDAK439ZK9,/pdbp_imagingmr_output_1634917064356/PDAK439ZK...,PDAK439ZK9_PDBP_ImagingMR_1634917112837_PDL_01...,V04
2,PDBP-DATA0029690,Baseline,PDJW615BUW,/pdbp_imagingmr_output_1539784418934/PDJW615BU...,PDJW615BUW_PDBP_ImagingMR_1539784418934,BL
3,PDBP-DATA0055727,12 months,PDLC368NK0,/pdbp_imagingmr_output_1634917064356/PDLC368NK...,PDLC368NK0_PDBP_ImagingMR_1634917108372_PDL_01...,V04
4,PDBP-DATA0055727,12 months,PDLC368NK0,/pdbp_imagingmr_output_1634917064356/PDLC368NK...,PDLC368NK0_PDBP_ImagingMR_1634917108479,V04
...,...,...,...,...,...,...
6094,PDBP-DATA0039491,Baseline,PDNF936JW4,/RawData-David Vaillancourt-ImagingMR/PDNF936J...,PDNF936JW4_ImagingMR_1427505427566_MSA_035_1,BL
6095,PDBP-DATA0039491,Baseline,PDNW868YJ6,/RawData-David Vaillancourt-ImagingMR/PDNW868Y...,PDNW868YJ6_ImagingMR_1428582370464_PDL_059_1,BL
6096,PDBP-DATA0061450,Baseline,NIHNE902MGFBR,/pdbp_imagingmr_output_1656028633008/NIHNE902M...,NIHNE902MGFBR_PDBP_ImagingMR_1656038998839,BL
6097,PDBP-DATA0061450,Baseline,NIHNE902MGFBR,/pdbp_imagingmr_output_1656028633008/NIHNE902M...,NIHNE902MGFBR_PDBP_ImagingMR_1656038973630,BL


In [21]:
PDBP_MRI_query['PATNO Visit']=None

C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_23580\2640799086.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PDBP_MRI_query['PATNO Visit']=None


In [22]:
for i in range(len(PDBP_MRI_query)):
    try:
        PDBP_MRI_query['PATNO Visit'][i]=PDBP_MRI_query['PDBP_ImagingMR.Main.GUID'][i]+'_'+PDBP_MRI_query['Visit'][i]
    except:
        print(i)

C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_23580\1978847872.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PDBP_MRI_query['PATNO Visit'][i]=PDBP_MRI_query['PDBP_ImagingMR.Main.GUID'][i]+'_'+PDBP_MRI_query['Visit'][i]


3081
3243
3244
3245
5155
5156
5157
5621
5970
5971
5972


In [23]:
PDBP_MRI_query['PATNO Visit'][0]

'PDAK439ZK9_V04'

In [24]:
PDBP_MRI_query

,Dataset,PDBP_ImagingMR.Main.VisitTypPDBP,PDBP_ImagingMR.Main.GUID,PDBP_ImagingMR.Image Information.ImgFile,Image ID,Visit,PATNO Visit
0,PDBP-DATA0055727,12 months,PDAK439ZK9,/pdbp_imagingmr_output_1634917064356/PDAK439ZK...,PDAK439ZK9_PDBP_ImagingMR_1634917112740_PDL_01...,V04,PDAK439ZK9_V04
1,PDBP-DATA0055727,12 months,PDAK439ZK9,/pdbp_imagingmr_output_1634917064356/PDAK439ZK...,PDAK439ZK9_PDBP_ImagingMR_1634917112837_PDL_01...,V04,PDAK439ZK9_V04
2,PDBP-DATA0029690,Baseline,PDJW615BUW,/pdbp_imagingmr_output_1539784418934/PDJW615BU...,PDJW615BUW_PDBP_ImagingMR_1539784418934,BL,PDJW615BUW_BL
3,PDBP-DATA0055727,12 months,PDLC368NK0,/pdbp_imagingmr_output_1634917064356/PDLC368NK...,PDLC368NK0_PDBP_ImagingMR_1634917108372_PDL_01...,V04,PDLC368NK0_V04
4,PDBP-DATA0055727,12 months,PDLC368NK0,/pdbp_imagingmr_output_1634917064356/PDLC368NK...,PDLC368NK0_PDBP_ImagingMR_1634917108479,V04,PDLC368NK0_V04
...,...,...,...,...,...,...,...
6094,PDBP-DATA0039491,Baseline,PDNF936JW4,/RawData-David Vaillancourt-ImagingMR/PDNF936J...,PDNF936JW4_ImagingMR_1427505427566_MSA_035_1,BL,PDNF936JW4_BL
6095,PDBP-DATA0039491,Baseline,PDNW868YJ6,/RawData-David Vaillancourt-ImagingMR/PDNW868Y...,PDNW868YJ6_ImagingMR_1428582370464_PDL_059_1,BL,PDNW868YJ6_BL
6096,PDBP-DATA0061450,Baseline,NIHNE902MGFBR,/pdbp_imagingmr_output_1656028633008/NIHNE902M...,NIHNE902MGFBR_PDBP_ImagingMR_1656038998839,BL,NIHNE902MGFBR_BL
6097,PDBP-DATA0061450,Baseline,NIHNE902MGFBR,/pdbp_imagingmr_output_1656028633008/NIHNE902M...,NIHNE902MGFBR_PDBP_ImagingMR_1656038973630,BL,NIHNE902MGFBR_BL


In [25]:
information_table

,Image ID,group,PATNO Visit
0,PDAA852XL7_PDBP_ImagingMR_1634919016780_PSP_02...,NaN,NaN
1,PDAA852XL7_PDBP_ImagingMR_1634919016887_PSP_02...,NaN,NaN
2,PDAA852XL7_PDBP_ImagingMR_1634921322116_PSP_02...,NaN,NaN
3,PDAA852XL7_PDBP_ImagingMR_1634921322227_PSP_02...,NaN,NaN
4,PDAD926WRZ_PDBP_ImagingMR_1634911430459_PDL_00...,NaN,NaN
...,...,...,...
708,PDZP161DT6_PDBP_ImagingMR_1634921352197_PSP_03...,NaN,NaN
709,PDZP161DT6_PDBP_ImagingMR_1634921352309_PSP_03...,NaN,NaN
710,PDZT233BE7_PDBP_ImagingMR_1634915165074_PDL_02...,NaN,NaN
711,PDZT233BE7_PDBP_ImagingMR_1634915165174_PDL_02...,NaN,NaN


In [26]:
PDBP_MRI_query[PDBP_MRI_query['Image ID']==information_table['Image ID'][0]]

,Dataset,PDBP_ImagingMR.Main.VisitTypPDBP,PDBP_ImagingMR.Main.GUID,PDBP_ImagingMR.Image Information.ImgFile,Image ID,Visit,PATNO Visit
273,PDBP-DATA0055728,Baseline,PDAA852XL7,/pdbp_imagingmr_output_1634918938130/PDAA852XL...,PDAA852XL7_PDBP_ImagingMR_1634919016780_PSP_02...,BL,PDAA852XL7_BL


In [27]:
PDBP_MRI_query[PDBP_MRI_query['Image ID']==information_table['Image ID'][5]]['PATNO Visit'].values[0]

'PDAD926WRZ_BL'

In [28]:
for i in range(len(information_table)):
    information_table['PATNO Visit'][i]=PDBP_MRI_query[PDBP_MRI_query['Image ID']==information_table['Image ID'][i]]['PATNO Visit'].values[0]

In [29]:
information_table

,Image ID,group,PATNO Visit
0,PDAA852XL7_PDBP_ImagingMR_1634919016780_PSP_02...,NaN,PDAA852XL7_BL
1,PDAA852XL7_PDBP_ImagingMR_1634919016887_PSP_02...,NaN,PDAA852XL7_BL
2,PDAA852XL7_PDBP_ImagingMR_1634921322116_PSP_02...,NaN,PDAA852XL7_V04
3,PDAA852XL7_PDBP_ImagingMR_1634921322227_PSP_02...,NaN,PDAA852XL7_V04
4,PDAD926WRZ_PDBP_ImagingMR_1634911430459_PDL_00...,NaN,PDAD926WRZ_BL
...,...,...,...
708,PDZP161DT6_PDBP_ImagingMR_1634921352197_PSP_03...,NaN,PDZP161DT6_V04
709,PDZP161DT6_PDBP_ImagingMR_1634921352309_PSP_03...,NaN,PDZP161DT6_V04
710,PDZT233BE7_PDBP_ImagingMR_1634915165074_PDL_02...,NaN,PDZT233BE7_BL
711,PDZT233BE7_PDBP_ImagingMR_1634915165174_PDL_02...,NaN,PDZT233BE7_BL


In [30]:
label_information=pd.read_csv('../MDS-UPDRS_2024-03-18T04-37-33/UPDRSIII_processed.csv')

In [31]:
label_information

,Study ID,Dataset,MDS_UPDRS.Required Fields.SiteName,MDS_UPDRS.Required Fields.VisitTypPDBP,MDS_UPDRS.Required Fields.VisitDate,MDS_UPDRS.Required Fields.GUID,MDS_UPDRS.Required Fields.ASSOCIATED GUID,MDS_UPDRS.Required Fields.AgeYrs,MDS_UPDRS.Required Fields.AgeRemaindrMonths,MDS_UPDRS.Required Fields.AgeVal,...,MDS_UPDRS.Part IV: Motor Complications.MDSUPDRSFuncImpactFluctScore,MDS_UPDRS.Part IV: Motor Complications.MDSUPDRSComplxtyMtrFluctScore,MDS_UPDRS.Part IV: Motor Complications.MDSUPDRSPainflOffStatDystnaScr,MDS_UPDRS.Part IV: Motor Complications.MDSUPDRSTtlHrOffDemnDystniaNum,MDS_UPDRS.Part IV: Motor Complications.MDSUPDRSTtlHrOffWDystniaNum,MDS_UPDRS.Part IV: Motor Complications.MDSUPDRSPrcntOffDystniaVal,MDS_UPDRS.Part IV: Motor Complications.MDSUPDRS_PartIVScore,MDS_UPDRS.Total Score.MDSUPDRS_TotalScore,PATNO Visit,Visit
0,206,PDBP-DATA0019553,UT-Southwestern Medical Center,48 months,NaN,PDDR568DKC,NaN,70 - 79,NaN,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,1.0,42.0,PDDR568DKC_V10,V10
1,206,PDBP-DATA0019553,UT-Southwestern Medical Center,36 months,NaN,PDYP417JN5,NaN,50 - 59,NaN,NaN,...,0.0,4.0,4.0,4.0,4.0,100.0,9.0,38.0,PDYP417JN5_V08,V08
2,206,PDBP-DATA0019553,UT-Southwestern Medical Center,42 months,NaN,PDTE038EAT,NaN,70 - 79,NaN,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,30.0,PDTE038EAT_V09,V09
3,206,PDBP-DATA0019553,UT-Southwestern Medical Center,48 months,NaN,PDHK041UGZ,NaN,40 - 49,NaN,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,33.0,PDHK041UGZ_V10,V10
4,206,PDBP-DATA0019553,UT-Southwestern Medical Center,42 months,NaN,PDUW285FH4,NaN,70 - 79,NaN,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,4.0,78.0,PDUW285FH4_V09,V09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9622,206,PDBP-DATA0010969,UT-Southwestern Medical Center,30 months,NaN,PDAB549YWB,NaN,50 - 59,NaN,NaN,...,3.0,3.0,4.0,7.0,7.0,100.0,13.0,37.0,PDAB549YWB_V07,V07
9623,206,PDBP-DATA0010969,UT-Southwestern Medical Center,18 months,NaN,PDXX391GDU,NaN,70 - 79,NaN,NaN,...,1.0,1.0,0.0,NaN,NaN,NaN,3.0,25.0,PDXX391GDU_V05,V05
9624,206,PDBP-DATA0012706,UT-Southwestern Medical Center,6 months,NaN,PDGE467UZT,NaN,70 - 79,NaN,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,41.0,PDGE467UZT_V02,V02
9625,206,PDBP-DATA0012706,UT-Southwestern Medical Center,Baseline,NaN,PDGZ943TUT,NaN,40 - 49,NaN,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,1.0,PDGZ943TUT_BL,BL


In [32]:
information_table

,Image ID,group,PATNO Visit
0,PDAA852XL7_PDBP_ImagingMR_1634919016780_PSP_02...,NaN,PDAA852XL7_BL
1,PDAA852XL7_PDBP_ImagingMR_1634919016887_PSP_02...,NaN,PDAA852XL7_BL
2,PDAA852XL7_PDBP_ImagingMR_1634921322116_PSP_02...,NaN,PDAA852XL7_V04
3,PDAA852XL7_PDBP_ImagingMR_1634921322227_PSP_02...,NaN,PDAA852XL7_V04
4,PDAD926WRZ_PDBP_ImagingMR_1634911430459_PDL_00...,NaN,PDAD926WRZ_BL
...,...,...,...
708,PDZP161DT6_PDBP_ImagingMR_1634921352197_PSP_03...,NaN,PDZP161DT6_V04
709,PDZP161DT6_PDBP_ImagingMR_1634921352309_PSP_03...,NaN,PDZP161DT6_V04
710,PDZT233BE7_PDBP_ImagingMR_1634915165074_PDL_02...,NaN,PDZT233BE7_BL
711,PDZT233BE7_PDBP_ImagingMR_1634915165174_PDL_02...,NaN,PDZT233BE7_BL


In [33]:
label_information[label_information['PATNO Visit']==information_table['PATNO Visit'][3]]

,Study ID,Dataset,MDS_UPDRS.Required Fields.SiteName,MDS_UPDRS.Required Fields.VisitTypPDBP,MDS_UPDRS.Required Fields.VisitDate,MDS_UPDRS.Required Fields.GUID,MDS_UPDRS.Required Fields.ASSOCIATED GUID,MDS_UPDRS.Required Fields.AgeYrs,MDS_UPDRS.Required Fields.AgeRemaindrMonths,MDS_UPDRS.Required Fields.AgeVal,...,MDS_UPDRS.Part IV: Motor Complications.MDSUPDRSFuncImpactFluctScore,MDS_UPDRS.Part IV: Motor Complications.MDSUPDRSComplxtyMtrFluctScore,MDS_UPDRS.Part IV: Motor Complications.MDSUPDRSPainflOffStatDystnaScr,MDS_UPDRS.Part IV: Motor Complications.MDSUPDRSTtlHrOffDemnDystniaNum,MDS_UPDRS.Part IV: Motor Complications.MDSUPDRSTtlHrOffWDystniaNum,MDS_UPDRS.Part IV: Motor Complications.MDSUPDRSPrcntOffDystniaVal,MDS_UPDRS.Part IV: Motor Complications.MDSUPDRS_PartIVScore,MDS_UPDRS.Total Score.MDSUPDRS_TotalScore,PATNO Visit,Visit
1595,202,PDBP-DATA0002755,University of Florida (Gainesville),12 months,NaN,PDAA852XL7,NaN,80 - 89,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PDAA852XL7_V04,V04


In [34]:
for i in range(len(information_table)):
    try:
        information_table['group'][i]=label_information[label_information['PATNO Visit']==information_table['PATNO Visit'][i]]['MDS_UPDRS.Part III: Motor Examination.MDSUPDRSHoehnYahrStageScore'].values[0].astype(int)
    except:
        print(i)

6
7
10
11
14
15
22
23
48
49


60
61
64
65
73
74
114
115
128
129
134
135
138
139
150
151
154
155
162
188
189
192
193
200
203
204
207
208
211
212
223
224
249
250
255
256
271
272
275
276
291
292
295
296
307
308
325
326
329
330
333
334
337
338
350
351
361
412
415
416
459
460
474
475
478
479
486
487
490
491
513
514
520
521
526
527
535
554
555
566
567
577
578
585
586
593
636
637
644
645
656
657
660
661
670
671
678
679
688
689
692
693
712


In [35]:
information_table

,Image ID,group,PATNO Visit
0,PDAA852XL7_PDBP_ImagingMR_1634919016780_PSP_02...,0,PDAA852XL7_BL
1,PDAA852XL7_PDBP_ImagingMR_1634919016887_PSP_02...,0,PDAA852XL7_BL
2,PDAA852XL7_PDBP_ImagingMR_1634921322116_PSP_02...,3,PDAA852XL7_V04
3,PDAA852XL7_PDBP_ImagingMR_1634921322227_PSP_02...,3,PDAA852XL7_V04
4,PDAD926WRZ_PDBP_ImagingMR_1634911430459_PDL_00...,2,PDAD926WRZ_BL
...,...,...,...
708,PDZP161DT6_PDBP_ImagingMR_1634921352197_PSP_03...,5,PDZP161DT6_V04
709,PDZP161DT6_PDBP_ImagingMR_1634921352309_PSP_03...,5,PDZP161DT6_V04
710,PDZT233BE7_PDBP_ImagingMR_1634915165074_PDL_02...,2,PDZT233BE7_BL
711,PDZT233BE7_PDBP_ImagingMR_1634915165174_PDL_02...,2,PDZT233BE7_BL


In [36]:
information_table['group'].value_counts()

0    249
2    184
1     50
3     43
4     43
5     31
Name: group, dtype: int64

In [37]:
249+184+50+43+43+31

600

In [37]:
# # max_count=0
# pre_ID=''
# count=0
# num=0
# for i in range(len(information_table)):
#     if information_table['PATNO Visit'][i] == pre_ID:
#         count+=1
#         # if count>max_count:
#         #     max_count=count
#         # m2['PATNO Visit'][i]=m2['PATNO Visit'][i]+f'_{count}'
#         information_table.drop(i, inplace=True)
#     else:
#         count=0
#         num+=1
#         pre_ID=information_table['PATNO Visit'][i]

In [38]:
information_table

,Image ID,group,PATNO Visit
0,PDAA852XL7_PDBP_ImagingMR_1634919016780_PSP_02...,0,PDAA852XL7_BL
1,PDAA852XL7_PDBP_ImagingMR_1634919016887_PSP_02...,0,PDAA852XL7_BL
2,PDAA852XL7_PDBP_ImagingMR_1634921322116_PSP_02...,3,PDAA852XL7_V04
3,PDAA852XL7_PDBP_ImagingMR_1634921322227_PSP_02...,3,PDAA852XL7_V04
4,PDAD926WRZ_PDBP_ImagingMR_1634911430459_PDL_00...,2,PDAD926WRZ_BL
...,...,...,...
708,PDZP161DT6_PDBP_ImagingMR_1634921352197_PSP_03...,5,PDZP161DT6_V04
709,PDZP161DT6_PDBP_ImagingMR_1634921352309_PSP_03...,5,PDZP161DT6_V04
710,PDZT233BE7_PDBP_ImagingMR_1634915165074_PDL_02...,2,PDZT233BE7_BL
711,PDZT233BE7_PDBP_ImagingMR_1634915165174_PDL_02...,2,PDZT233BE7_BL


In [39]:
information_table.dropna(inplace=True)

In [40]:
information_table

,Image ID,group,PATNO Visit
0,PDAA852XL7_PDBP_ImagingMR_1634919016780_PSP_02...,0,PDAA852XL7_BL
1,PDAA852XL7_PDBP_ImagingMR_1634919016887_PSP_02...,0,PDAA852XL7_BL
2,PDAA852XL7_PDBP_ImagingMR_1634921322116_PSP_02...,3,PDAA852XL7_V04
3,PDAA852XL7_PDBP_ImagingMR_1634921322227_PSP_02...,3,PDAA852XL7_V04
4,PDAD926WRZ_PDBP_ImagingMR_1634911430459_PDL_00...,2,PDAD926WRZ_BL
...,...,...,...
707,PDZP161DT6_PDBP_ImagingMR_1634921243650_PSP_03...,5,PDZP161DT6_BL
708,PDZP161DT6_PDBP_ImagingMR_1634921352197_PSP_03...,5,PDZP161DT6_V04
709,PDZP161DT6_PDBP_ImagingMR_1634921352309_PSP_03...,5,PDZP161DT6_V04
710,PDZT233BE7_PDBP_ImagingMR_1634915165074_PDL_02...,2,PDZT233BE7_BL


In [41]:
meta_all = pd.DataFrame(columns = ["img_array","label","PATNO Visit"])

In [42]:
information_table[information_table['group'].notna()]['Image ID'].values.tolist()

['PDAA852XL7_PDBP_ImagingMR_1634919016780_PSP_021_1_sT1W_3D_TFE_SENSE_5_1',
 'PDAA852XL7_PDBP_ImagingMR_1634919016887_PSP_021_1_T2W_VISTA_HR_SENSE_9_1',
 'PDAA852XL7_PDBP_ImagingMR_1634921322116_PSP_021_2_sT1W_3D_TFE_SENSE_5_1',
 'PDAA852XL7_PDBP_ImagingMR_1634921322227_PSP_021_2_T2W_VISTA_HR_SENSE_10_1',
 'PDAD926WRZ_PDBP_ImagingMR_1634911430459_PDL_004_sT1W_3D_TFE_SENSE_7_1',
 'PDAD926WRZ_PDBP_ImagingMR_1634911430575_PDL_004_T2W_VISTA_HR_SENSE_8_1',
 'PDAH499RDX_PDBP_ImagingMR_1634915227941_PDL_031_1_sT1W_3D_TFE_SENSE_6_1',
 'PDAH499RDX_PDBP_ImagingMR_1634915228042_PDL_031_1_T2W_VISTA_HR_SENSE_13_1',
 'PDAK439ZK9_PDBP_ImagingMR_1634915138629_PDL_18_1_sT1W_3D_TFE_SENSE_6_1',
 'PDAK439ZK9_PDBP_ImagingMR_1634915138730_PDL_18_1_T2W_VISTA_HR_SENSE_10_1',
 'PDAL812VRR_ImagingMR_1427589757794_ET_023_1',
 'PDAL812VRR_ImagingMR_1427589759085_ET_023_1',
 'PDAL812VRR_ImagingMR_1427589766113_ET_023_2',
 'PDAL812VRR_ImagingMR_1427589770489_ET_023_2',
 'PDAN014LKF_PDBP_ImagingMR_1634906776184_CON_

In [43]:
len(information_table[information_table['group'].notna()]['Image ID'].values.tolist())

600

In [44]:
def create_dataset(meta, meta_all,path_to_datadir):
    files = os.listdir(path_to_datadir)
    end = '.nii'
    count=0
    meta_list=meta[meta['group'].notna()]['Image ID'].values.tolist()
    print(meta_list)
    for file in files:
        # print(file)
        print(count)
        count+=1
        if file != '.DS_Store':
            path = os.path.join(path_to_datadir, file)
            # print(path)
            img_id = file.split('.')[0]
            # print(img_id)
            if img_id in meta_list:
                idx = meta[meta["Image ID"] == img_id].index[0]
                # print(idx)
                im = nib.load(path).get_fdata()
                # # # print(im.shape)
                n_i, n_j, n_k = im.shape
                center_i = (n_i - 1) // 2
                center_j = (n_j - 1) // 2
                center_k = (n_k - 1) // 2
                im1 = skTrans.resize(im[center_i, :, :], (72, 72), order=1, preserve_range=True)
                im2 = skTrans.resize(im[:, center_j, :], (72, 72), order=1, preserve_range=True)
                im3 = skTrans.resize(im[:, :, center_k], (72, 72), order=1, preserve_range=True)
                im = np.array([im1,im2,im3])
                label = meta.at[idx, "group"]
                subject = meta.at[idx, "PATNO Visit"]
                # norm_im = normalize_img(im)
                new_row = pd.DataFrame({"img_array": [im], "label": [label], "PATNO Visit": [subject]})
                meta_all = pd.concat([meta_all, new_row], ignore_index=True)

    meta_all.to_pickle("MRI_processed_6class_nobrain/mri_meta_2D_3slice_nobrain.pkl")
    # meta_all.flush()
    # os.fsync(meta_all.fileno())
    # time.sleep(0.5)

In [45]:
#wjk-AD三分类-2文件夹中内容为wjk-AD三分类中的所有nii文件（不分AD、NC、MCI文件夹）
create_dataset(information_table, meta_all, 'MRI_nobrain/')

['PDAA852XL7_PDBP_ImagingMR_1634919016780_PSP_021_1_sT1W_3D_TFE_SENSE_5_1', 'PDAA852XL7_PDBP_ImagingMR_1634919016887_PSP_021_1_T2W_VISTA_HR_SENSE_9_1', 'PDAA852XL7_PDBP_ImagingMR_1634921322116_PSP_021_2_sT1W_3D_TFE_SENSE_5_1', 'PDAA852XL7_PDBP_ImagingMR_1634921322227_PSP_021_2_T2W_VISTA_HR_SENSE_10_1', 'PDAD926WRZ_PDBP_ImagingMR_1634911430459_PDL_004_sT1W_3D_TFE_SENSE_7_1', 'PDAD926WRZ_PDBP_ImagingMR_1634911430575_PDL_004_T2W_VISTA_HR_SENSE_8_1', 'PDAH499RDX_PDBP_ImagingMR_1634915227941_PDL_031_1_sT1W_3D_TFE_SENSE_6_1', 'PDAH499RDX_PDBP_ImagingMR_1634915228042_PDL_031_1_T2W_VISTA_HR_SENSE_13_1', 'PDAK439ZK9_PDBP_ImagingMR_1634915138629_PDL_18_1_sT1W_3D_TFE_SENSE_6_1', 'PDAK439ZK9_PDBP_ImagingMR_1634915138730_PDL_18_1_T2W_VISTA_HR_SENSE_10_1', 'PDAL812VRR_ImagingMR_1427589757794_ET_023_1', 'PDAL812VRR_ImagingMR_1427589759085_ET_023_1', 'PDAL812VRR_ImagingMR_1427589766113_ET_023_2', 'PDAL812VRR_ImagingMR_1427589770489_ET_023_2', 'PDAN014LKF_PDBP_ImagingMR_1634906776184_CON_015_1_sT1W_3D_

In [2]:
#reading in a dataframe that contains image arrays, patient IDs ("subject"), and diagnosis
m2 = pd.read_pickle("MRI_processed_6class_nobrain/mri_meta_2D_3slice_nobrain.pkl")

In [3]:
m2

,img_array,label,PATNO Visit
0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0,PDAA852XL7_BL
1,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0,PDAA852XL7_BL
2,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",3,PDAA852XL7_V04
3,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",3,PDAA852XL7_V04
4,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,PDAD926WRZ_BL
...,...,...,...
595,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",5,PDZP161DT6_BL
596,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",5,PDZP161DT6_V04
597,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",5,PDZP161DT6_V04
598,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,PDZT233BE7_BL


In [6]:
m2['PATNO Visit'].values

array(['PDAA852XL7_BL', 'PDAA852XL7_BL', 'PDAA852XL7_V04',
       'PDAA852XL7_V04', 'PDAD926WRZ_BL', 'PDAD926WRZ_BL',
       'PDAH499RDX_BL', 'PDAH499RDX_BL', 'PDAK439ZK9_BL', 'PDAK439ZK9_BL',
       'PDAL812VRR_BL', 'PDAL812VRR_BL', 'PDAL812VRR_V04',
       'PDAL812VRR_V04', 'PDAN014LKF_BL', 'PDAN014LKF_BL',
       'PDAP978ED4_BL', 'PDAP978ED4_BL', 'PDAP978ED4_V04',
       'PDAP978ED4_V04', 'PDAZ576HCR_BL', 'PDAZ576HCR_BL',
       'PDAZ576HCR_BL', 'PDAZ576HCR_BL', 'PDAZ576HCR_V04',
       'PDAZ576HCR_V04', 'PDBG857AZQ_BL', 'PDBG857AZQ_BL',
       'PDBG857AZQ_V04', 'PDBG857AZQ_V04', 'PDBH812DN5_BL',
       'PDBH812DN5_BL', 'PDBH812DN5_BL', 'PDBH812DN5_BL', 'PDBH812DN5_BL',
       'PDBH812DN5_V04', 'PDBH812DN5_V04', 'PDBH812DN5_BL',
       'PDBJ362RKM_BL', 'PDBJ362RKM_BL', 'PDBK961FJ1_BL', 'PDBK961FJ1_BL',
       'PDBM325HN9_BL', 'PDBM325HN9_BL', 'PDBM325HN9_BL', 'PDBM325HN9_BL',
       'PDBM325HN9_V04', 'PDBM325HN9_V04', 'PDBN442LGY_BL',
       'PDBN442LGY_BL', 'PDBR124KBG_BL', 'PDBR12

In [50]:
# max_count=0
pre_ID=''
count=0
num=0
for i in range(len(m2)):
    if m2['PATNO Visit'][i] == pre_ID:
        count+=1
        # if count>max_count:
        #     max_count=count
        # m2['PATNO Visit'][i]=m2['PATNO Visit'][i]+f'_{count}'
        # m2.drop(i, inplace=True)
    else:
        count=0
        num+=1
        pre_ID=m2['PATNO Visit'][i]

In [51]:
num

270

In [52]:
m2

,img_array,label,PATNO Visit
0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0,PDAA852XL7_BL
1,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",3,PDAA852XL7_V04
2,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,PDAD926WRZ_BL
3,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,PDAH499RDX_BL
4,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,PDAK439ZK9_BL
...,...,...,...
265,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0,PDZN621NR3_BL
266,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0,PDZN621NR3_V04
267,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",5,PDZP161DT6_BL
268,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",5,PDZP161DT6_V04


In [48]:
m2['label'].value_counts()

0    249
2    184
1     50
3     43
4     43
5     31
Name: label, dtype: int64